In [ ]:
import os
import cv2
import numpy as np
import json
import random

LOAD_FISH_EYE_CONFIG = False

VIDEO_INPUT = {
    'width': 1920,
    'height': 1080
}

DIM = (1920, 1080)
K = np.array(
    [[1339.2898532184101, 0.0, 912.605852159453], [0.0, 1334.6227716562116, 540.699525078661], [0.0, 0.0, 1.0]])
D = np.array([[-0.019105044374337198], [0.26447332249046934], [-0.7164055812838268], [0.6830070634869604]])

FRAME_COUNT = 0
BASE_VIDEO_INPUT_PATH = './undistorted_cameradata/'
BASE_OUTPUT_PATH = './undistorted_cameradata/'

VIDEO_INPUTS = {
    '3': 'camera_3.mp4',
    #'4': 'Camera4_2018-08-28_14h25min40s587ms.mp4',
    #'5': 'Camera5_2018-08-28_14h25min41s707ms.mp4',
    #'6': 'Camera6_2018-08-28_14h26min06s927ms.mp4',
    #'7': 'Camera7_2018-08-28_14h25min42s480ms.mp4',
    #'8': 'Camera8_2018-08-28_14h25min43s537ms.mp4'
}

info = []
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)

# cap.set(1, 1843)

try:
    os.mkdir(BASE_OUTPUT_PATH)
except FileExistsError:
    pass

for camera, camera_name in VIDEO_INPUTS.items():
    cap = cv2.VideoCapture(BASE_VIDEO_INPUT_PATH + camera_name)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_numbers = [random.randint(1, length - 1) for i in range(100)]
    frame_numbers = sorted(frame_numbers)
    frame_numbers = np.unique(frame_numbers)

    for frame_number in frame_numbers:

        cap.set(1, frame_number)

        has_next, frame = cap.read()

        file_name = 'camera_{}_frame_{}.jpg'.format(camera, frame_number)
        file_path = 'labeled_data/raw_frames/' + file_name

        cv2.imwrite(file_path, frame)

        info.append({
            'frame_number': int(frame_number),
            'camera': int(camera),
            'width': width,
            'height': height,
            'file_path': file_path,
            'file_name': file_name
        })

    cap.release()

with open('labeled_data/raw_frames_info.json', 'w') as outfile:
    json.dump(info, outfile)


